# Applied Data Science Capstone - Nathan Anderson

# Segmenting and Clustering Neighborhoods in Toronto 

## Step 3

### Importing Libraries

In [302]:
# !conda install -c conda-forge geopy --yes        
# !conda install -c conda-forge folium=0.5.0 --yes 
# !conda install -c conda-forge geopy --yes
# !conda install -c conda-forge folium=0.5.0 --yes 

import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

### Import CSV File from Step 2

In [303]:
to_csv = "Toronto.Step2_df.csv"
to_neighborhoods = pd.read_csv(to_csv)
to_neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


In [304]:
# Function that will retrieve Latitude and Longitude from Postalcode
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

postal_codes = df['Postalcode']
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

# Add Lat and Long Data
df_latlong = pd.DataFrame(coords, columns = ['Latitude', 'Longitude'])
df['Latitude'] = df_latlong['Latitude']
df['Longitude'] = df_latlong['Longitude']

df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.726881,-79.265694
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.713340,-79.284942
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.723538,-79.228353
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696448,-79.265642


### Use Geopy Library to get the latitude and longitude values of Toronto

In [305]:
address = "Toronto, Ontario"

geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto, Ontario are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top

In [306]:
# Create the map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# Add markers
for lat, lng, borough, neighborhood in zip(to_neighborhoods['Latitude'], to_neighborhoods['Longitude'], to_neighborhoods['Borough'], to_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### Find Boroughs that contain the word 'Toronto'

In [307]:
to_boroughs = to_neighborhoods[to_neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop = True)
to_boroughs.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.678148,-79.295349
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578
3,M4M,East Toronto,Studio District,43.648000,-79.339260
4,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415


### Update map with Boroughs containg the word 'Toronto'

In [308]:
# Create the map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# Add markers
for lat, lng, label in zip(to_boroughs['Latitude'], to_boroughs['Longitude'], to_boroughs['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### Define Foursquare Credentials and Version

In [309]:
CLIENT_ID = '5VN3HJ4F0FMK1420GNLRH5CFJZSZ5YN2KYYQ0HQQK2QC3UN4' 
CLIENT_SECRET = '2D45AJOKLT2SBE5CGAJUTCRUX0XSWU1B5G5F10J3I1XVQJMY' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5VN3HJ4F0FMK1420GNLRH5CFJZSZ5YN2KYYQ0HQQK2QC3UN4
CLIENT_SECRET:2D45AJOKLT2SBE5CGAJUTCRUX0XSWU1B5G5F10J3I1XVQJMY


### Exploring one neighborhood

In [310]:
# Chossing a Neighborhood
to_boroughs.loc[0, 'Neighborhood']

'The Beaches'

In [311]:
# Get the Latitude and Longitude for the chosed Neighborhood
neighborhood_latitude = to_boroughs.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = to_boroughs.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = to_boroughs.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67814827600006, -79.29534930999995.


### Retrieving the top 100 Venues

In [312]:
LIMIT = 100 
radius = 500 
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ecec5f478a484001b17cf10'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.68264828050006,
    'lng': -79.28913883913675},
   'sw': {'lat': 43.673648271500056, 'lng': -79.30155978086314}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

### Function that extracts the category of the venue

In [313]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list)  == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the data and structure it into a pandas dataframe

In [314]:
venues = results['response']['groups'][0]['items']

nearby_venues = pd.json_normalize(venues)

# Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues[filtered_columns]

# Filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# Clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Calvary Baptist Church,Church,43.681059,-79.299246


In [315]:
# How many venues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


### A function to repeat the same process for all the neighborhoods

In [316]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [317]:
toronto_venues = getNearbyVenues(names=to_neighborhoods['Neighborhood'],
                                   latitudes=to_neighborhoods['Latitude'],
                                   longitudes=to_neighborhoods['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [318]:
print(toronto_venues.shape)
toronto_venues.head()

(2280, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.808626,-79.189913,Upper Rouge Trail,43.809988,-79.186147,Trail
1,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
3,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
4,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Swiss Chalet,43.768122,-79.190493,Restaurant


### Analyze each Neighborhood

In [319]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back into dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]



### Group the rows by neighborhood and by taking the mean of the frequency of occurence of each category

In [320]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cable Car,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000

In [321]:
toronto_grouped.shape

(97, 267)

### Printing each neighborhood along with the top 5 most common venues

In [322]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [323]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Sushi Restaurant,Badminton Court,Skating Rink,Supermarket,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant
1,"Alderwood, Long Branch",Athletics & Sports,Convenience Store,Pub,Coffee Shop,Gas Station,Gym,Sandwich Place,Pharmacy,Pizza Place,Falafel Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Ice Cream Shop,Shopping Mall,Sandwich Place,Middle Eastern Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint,Diner
3,Bayview Village,Trail,Construction & Landscaping,Women's Store,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Sushi Restaurant,Coffee Shop,Thai Restaurant,Indian Restaurant,Comfort Food Restaurant,Liquor Store,Butcher,Sports Club


### Cluster Neighborhoods - Run k-means to cluster neighborhood into 5 clusters

In [324]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 2, 2, 4, 1, 2, 1, 1, 2, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1])

In [346]:
# add clustering lables
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = to_boroughs

# merge data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
toronto_merged = toronto_merged.dropna()
toronto_merged.head(39)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.678148,-79.295349,1.0,Pub,Trail,Health Food Store,Church,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683424,-79.354564,1.0,Grocery Store,Discount Store,Business Service,Park,Bus Line,Women's Store,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668291,-79.315578,1.0,Fast Food Restaurant,Park,Fish & Chips Shop,Ice Cream Shop,Burrito Place,Liquor Store,Light Rail Station,Board Shop,Steakhouse,Sandwich Place
3,M4M,East Toronto,Studio District,43.648000,-79.339260,1.0,Night Market,Baseball Field,Government Building,Business Service,Women's Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School
4,M4N,Central Toronto,Lawrence Park,43.729455,-79.386415,1.0,Swim School,Bus Line,Women's Store,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Food
5,M4P,Central Toronto,Davisville North,43.713171,-79.388870,1.0,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym / Fitness Center,Gym,Women's Store,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.714139,-79.406456,2.0,Playground,Garden,Gym Pool,Park,Elementary School,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
7,M4S,Central Toronto,Davisville,43.703327,-79.385649,1.0,Dessert Shop,Sandwich Place,Café,Pizza Place,Italian Restaurant,Coffee Shop,Park,Thai Restaurant,Fast Food Restaurant,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690328,-79.383522,2.0,Playground,Convenience Store,Park,Gym,Restaurant,Women's Store,Electronics Store,Dog Run,Donut Shop,Dumpling Restaurant
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686378,-79.402372,1.0,Light Rail Station,Coffee Shop,Supermarket,Liquor Store,Skating Rink,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant


In [350]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

### Visualize the Clusters

In [351]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Cluster 1

In [355]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1.0], toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]

(   Postalcode           Borough  \
 0         M4E      East Toronto   
 1         M4K      East Toronto   
 2         M4L      East Toronto   
 3         M4M      East Toronto   
 4         M4N   Central Toronto   
 5         M4P   Central Toronto   
 7         M4S   Central Toronto   
 9         M4V   Central Toronto   
 11        M4X  Downtown Toronto   
 12        M4Y  Downtown Toronto   
 13        M5A  Downtown Toronto   
 14        M5B  Downtown Toronto   
 15        M5C  Downtown Toronto   
 16        M5E  Downtown Toronto   
 17        M5G  Downtown Toronto   
 18        M5H  Downtown Toronto   
 19        M5J  Downtown Toronto   
 20        M5K  Downtown Toronto   
 21        M5L  Downtown Toronto   
 23        M5P   Central Toronto   
 24        M5R   Central Toronto   
 25        M5S  Downtown Toronto   
 26        M5T  Downtown Toronto   
 27        M5V  Downtown Toronto   
 28        M5W  Downtown Toronto   
 29        M5X  Downtown Toronto   
 30        M6G  Downtown Tor